# Regularized Linear Regression
<span style="font-size: 12px;">By: Marisol Hernandez</span>

## 1. Introduction to Regularized Linear Regression

Linear regression is a fundamental technique used for modeling the relationship between a target variable and one or more feature variables. However, traditional linear regression models can be prone to overfitting when dealing with datasets with high dimensionality or *\*multicollinearity*.

*\*Sidebar — multicollinearity is when two or more independent variables in a regression model are highly correlated with each other.*

Regularized linear regression techniques, such as **Ridge** and **Lasso** regression, address the issue of overfitting by introducing penalty terms to the cost function. These penalties discourage the model from fitting the data too closely, leading to better generalization performance.

<p align="center">
  <img src="imgs/regularization_impact.png" alt="Alt text" width="550" height="300">
</p>

## 2. Multiple Linear Regression without Regularization
Before exploring the effects of regularization, we want to model a multiple linear regression that can be expressed as:

$$\hat{y} = w_0 + w_1x_1 + w_2x_2 + ... + w_dx_d$$

where:

- $\hat{y}$ is the target variable

- $w_0$ corresponds to the y-intercept 

- $w_1, w_2, ..., w_d$ are the coefficients corresponding to each feature

- $x_1, x_2, ..., x_d$ are the features

Below we read in the **Airbnb in New York** dataset.

In [11]:
import pandas as pd

total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv")
total_data.head(5)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Each row in the dataset corresponds to a Airbnb rental in New York. The dataset contains features such as neighbourhood_group, neighbourhood, latitude, longitude, room_type, etc.

`price` represents the price of the Airbnb for a single night in USD. In other words, it is the taret that we aim to predict using the features (independent variables) provided in the dataset.

Though the focus of this notebook is not on EDA, we still need to perform some EDA and data preprocessing before we can model. First, lets drop any irrelevant features.



In [12]:
irrelevant_features = ["id", "name", "host_id", "host_name", "last_review"]

for feature in irrelevant_features:
    total_data = total_data.drop(columns=feature)

total_data.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,1,365
3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


Now lets separate our dataset into `X` and `y`.

In [13]:
# X includes all features excluding the target
X = total_data.drop(columns="price")

# Print X to confirm that price is no longer there
X.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,1,9,0.21,6,365
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,1,45,0.38,2,355
2,Manhattan,Harlem,40.80902,-73.94190,Private room,3,0,NaN,1,365
3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,1,270,4.64,1,194
4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,10,9,0.10,1,0


In [14]:
# y represents the target
y = total_data["price"]

# Print y to inspect
y

0        149
1        225
2        150
3         89
4         80
        ... 
48890     70
48891     40
48892    115
48893     55
48894     90
Name: price, Length: 48895, dtype: int64

The next step would be to factorize our categorical features (`neighbourhood_group`, `neighbourhood`, `room_type`). But first lets see if there are any missing values we need to address.

In [15]:
categorical_features = ["neighbourhood_group", "neighbourhood", "room_type"]

# Check for missing values in the specified columns
missing_values = X[categorical_features].isnull().sum()

# Print the result
print("Missing values in categorical features:")
print(missing_values)

Missing values in categorical features:
neighbourhood_group    0
neighbourhood          0
room_type              0
dtype: int64


No missing values:) Let's factorize. Usually we would do this before the train/test split, but instead lets prepare and fit a pipeline that does this so we can later use it after the train/test split and on new unseen data. This pipeline will be fit on the entire dataset since there is not risk for data leakage when factorizing categorical features.

*Note: A separate notebook on pipelines will be prepared.*

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

# Create a custom transformer class for factorizing categorical features
class FactorizeCategorical(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        for feature in self.categorical_features:
            # Factorize the feature
            X_copy[feature + '_factorized'], _ = pd.factorize(X_copy[feature])
            # Drop the original feature from the dataset
            X_copy.drop(columns=feature, inplace=True)
        return X_copy

categorical_pipeline = Pipeline([
    ('factorize_categorical', FactorizeCategorical(categorical_features))
])

# Fit the categorical pipeline on the entire dataset
categorical_pipeline.fit(X)

Pipeline(steps=[('factorize_categorical',
                 FactorizeCategorical(categorical_features=['neighbourhood_group',
                                                            'neighbourhood',
                                                            'room_type']))])

We still need to impute missing values/outliers and scale but first we need to do a train/test split to avoid data leakage.

In [17]:
from sklearn.model_selection import train_test_split

# Split the data into training (80% of the data) and testing (20% of the data) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In a previous session, we imputed missing values/outliers without the using pipelines. In this notebook lets use pipelines. 

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a custom transformer class for replacing outliers
class ReplaceOutliers(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # This loop will replace outliers for each column
        for column in X.columns:
            # Calculate Q_25 and Q_75 for the column
            Q_25 = X[column].quantile(0.25)
            Q_75 = X[column].quantile(0.75)

            # Calculate the IQR
            IQR = Q_75 - Q_25

            # Calculate the upper and lower limit
            upper_limit = Q_75 + 1.5 * IQR
            lower_limit = Q_25 - 1.5 * IQR

            # Replace the outliers
            X.loc[X[column] < lower_limit, column] = Q_25
            X.loc[X[column] > upper_limit, column] = Q_75

        return X

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer  # sklearn has a pre-built imputer for NaNs
from sklearn.preprocessing import StandardScaler

# Define my numeric features, we defined the categorical features earlier
numeric_features = ["latitude", "longitude", "minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365"]

# Create the pipeline for numeric features
numeric_pipeline = Pipeline([
    ('outlier_replacement', ReplaceOutliers()),
    ('nan_imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),  
])

# Combine the pipelines using ColumnTransformer
combined_pipeline = ColumnTransformer([
    ('categorical_transformer', categorical_pipeline, categorical_features),
    ('numeric_transformer', numeric_pipeline, numeric_features),
])

# Fit and transform X_train
X_train_transformed = combined_pipeline.fit_transform(X_train)

# Convert X_train to a DataFrame just for inspection
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=categorical_features + numeric_features)
X_train_transformed_df.head()

,neighbourhood_group,neighbourhood,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0.0,0.0,0.0,-0.246025,0.075660,-0.010503,-0.033096,-0.199970,-0.666634,-0.852862
1,1.0,1.0,1.0,2.289173,0.573997,-0.538561,-0.707001,-0.861482,-0.666634,-0.860447
2,0.0,2.0,1.0,-0.737101,-0.064561,-0.538561,-0.856758,-0.330409,1.008775,-0.860447
3,0.0,2.0,0.0,-0.844666,0.886719,-0.010503,0.940324,0.880812,-0.666634,1.164663
4,2.0,3.0,1.0,0.348435,2.081787,1.045613,0.116661,-0.777629,-0.666634,-0.860447


In [22]:
# Transform X_test
X_test_transformed = combined_pipeline.transform(X_test)

# Convert X_test to a DataFrame just for inspection
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=categorical_features + numeric_features)
X_test_transformed_df.head()

,neighbourhood_group,neighbourhood,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0.0,0.0,0.0,-1.618719,-0.679917,-0.010503,0.865445,-0.349043,-0.666634,0.573058
1,1.0,1.0,1.0,-0.419156,1.673231,1.045613,-0.856758,-0.330409,-0.666634,-0.306765
2,2.0,2.0,1.0,0.616588,-1.096543,-0.538561,0.416175,-0.609921,-0.666634,-0.860447
3,2.0,3.0,0.0,-0.400721,-1.780496,-0.538561,-0.482366,0.741056,1.008775,1.202586
4,2.0,4.0,0.0,1.289727,0.829891,-0.538561,1.389594,-0.265189,-0.666634,-0.632906


Now we can train our MLR and assess it using $\text{MSE}$.

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Fit the Multiple Linear Regression model
# Notice that we are using X_train_transformed to fit
mlr_model = LinearRegression()
mlr_model.fit(X_train_transformed, y_train)

# Make predictions using X_test_transformed
y_pred = mlr_model.predict(X_test_transformed)

# Calculate Mean Squared Error
mlr_mse = mean_squared_error(y_test, y_pred)

# Print the MSE
print("Mean Squared Error (Multiple Linear Regression):", mlr_mse)
print(f"We can say that our model's predictions are off by ${round(mlr_mse, 2)} on average")

Mean Squared Error (Multiple Linear Regression): 38842.574209144375
We can say that our model's predictions are off by $38842.57 on average


Now lets inspect the intercept and the coefficients of our model.

In [30]:
# Get the coefficients of the linear regression model
coefficients = mlr_model.coef_

# Get the intercept of the linear regression model
intercept = mlr_model.intercept_

# Create a DataFrame for the intercept
intercept_df = pd.DataFrame([intercept], index=['intercept'], columns=['Coefficient'])

# Convert coefficients array into a DataFrame for easier inspection
coefficients_df = pd.DataFrame(coefficients, index=X_train_transformed_df.columns, columns=['Coefficient'])

# Concatenate intercept DataFrame with coefficients DataFrame
mlr_coefficients_df = pd.concat([intercept_df, coefficients_df])

# Print the DataFrame
print("MLR Model Coefficients:")
print(mlr_coefficients_df)


MLR Model Coefficients:
                                Coefficient
intercept                        205.156811
neighbourhood_group                1.483442
neighbourhood                     -0.111259
room_type                        -97.708054
latitude                          11.499133
longitude                        -36.560105
minimum_nights                   -10.815672
number_of_reviews                -15.942762
reviews_per_month                 -2.620639
calculated_host_listings_count    -2.223115
availability_365                  27.028622


## 3. Ridge Regression
Ridge regression adds a penalty term proportional to the square of the magnitude of coefficients to the cost function (in this case, the cost function is $\text{MSE}$). This penalty term shrinks the coefficients towards zero, but they never actually reach zero.

Mathematically, the objective function of Ridge regression can be expressed as:

$$J(w) = \text{MSE} + \lambda \sum_{d=1}^{m} w_d^2$$

$$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~= \frac{1}{2n}\sum_{i=1}^{n}(\hat{y}_{i} - y_{i})^2 + \lambda \sum_{d=1}^{m} w_d^2$$

$$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~= \frac{1}{2n}\sum_{i=1}^{n}((w_0 + w_1x_{i1} + w_2x_{i2} + ... + w_dx_{id}) - y_{i})^2 + \lambda \sum_{d=1}^{m} w_d^2$$

where $\text{MSE}$ is the mean squared error, $\lambda$  is the regularization parameter, and $w_i$ are the regression coefficients.